In [1]:
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
from ultralytics import YOLO

In [3]:
# model = YOLO("yolo11m-pose.pt")
# model.export(format="engine")

In [3]:
ENGINE_PATH = "../data/models/yolo11m-pose.engine"
VIDEO_INPUT = "../data/3196221-uhd_3840_2160_25fps.mp4"
VIDEO_OUTPUT = "../data/yolo11m_pose_tracked.mp4"
CSV_OUTPUT = "../data/yolo11m_pose_tracked.csv"

In [4]:
POSE_CONNECTIONS = [
    (5, 7), (7, 9),      # left arm
    (6, 8), (8, 10),     # right arm
    (5, 6),              # shoulders
    (5, 11), (6, 12),    # torso
    (11, 13), (13, 15),  # left leg
    (12, 14), (14, 16)   # right leg
]

In [5]:
model = YOLO(ENGINE_PATH)

In [6]:
def draw_pose(frame, kpts, track_id):
    h, w, _ = frame.shape

    for x, y, conf in kpts:
        if conf > 0.3:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 0, 255), -1)

    for a, b in POSE_CONNECTIONS:
        if kpts[a][2] > 0.3 and kpts[b][2] > 0.3:
            x1, y1 = int(kpts[a][0]), int(kpts[a][1])
            x2, y2 = int(kpts[b][0]), int(kpts[b][1])
            cv2.line(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)

    if track_id is not None:
        cx, cy = int(kpts[0][0]), int(kpts[0][1])
        cv2.putText(
            frame, f"ID {track_id}",
            (cx, cy - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6, (255, 255, 0), 2
        )


In [7]:
cap = cv2.VideoCapture(VIDEO_INPUT)

fps = cap.get(cv2.CAP_PROP_FPS)
w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(VIDEO_OUTPUT, fourcc, fps, (w, h))


In [8]:
records = []

with tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), desc="Tracking") as pbar:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model.track(
            frame,
            persist=True,
            conf=0.4,
            iou=0.5,
            tracker="bytetrack.yaml",
            verbose=False
        )

        if results and results[0].keypoints is not None:
            boxes = results[0].boxes
            kpts = results[0].keypoints.xy.cpu().numpy()
            confs = results[0].keypoints.conf.cpu().numpy()

            track_ids = (
                boxes.id.cpu().numpy().astype(int)
                if boxes.id is not None else [None] * len(kpts)
            )

            for i, (kp, kp_conf) in enumerate(zip(kpts, confs)):
                person_id = track_ids[i]
                kpts_full = np.hstack([kp, kp_conf[:, None]])

                draw_pose(frame, kpts_full, person_id)

                row = {
                    "frame": int(cap.get(cv2.CAP_PROP_POS_FRAMES)),
                    "track_id": person_id
                }

                for j, (x, y, c) in enumerate(kpts_full):
                    row[f"kp{j}_x"] = x
                    row[f"kp{j}_y"] = y
                    row[f"kp{j}_conf"] = c

                records.append(row)
        print(type(results[0].keypoints), results[0].keypoints is None)
        out.write(frame)
        pbar.update(1)

cap.release()
out.release()


Tracking:   0%|          | 0/385 [00:00<?, ?it/s]

Loading ../data/models/yolo11m-pose.engine for TensorRT inference...


Tracking:   0%|          | 1/385 [00:10<1:09:39, 10.88s/it]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking:   1%|          | 4/385 [00:11<10:01,  1.58s/it]  

...

Tracking:  99%|█████████▉| 382/385 [01:25<00:00,  3.79it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|█████████▉| 384/385 [01:26<00:00,  4.90it/s]

<class 'ultralytics.engine.results.Keypoints'> False
<class 'ultralytics.engine.results.Keypoints'> False


Tracking: 100%|██████████| 385/385 [01:26<00:00,  4.46it/s]


In [9]:
df = pd.DataFrame(records)
df.to_csv(CSV_OUTPUT, index=False)

print("Done.")
print("Video:", VIDEO_OUTPUT)
print("CSV:", CSV_OUTPUT)


Done.
Video: ../data/yolo11m_pose_tracked.mp4
CSV: ../data/yolo11m_pose_tracked.csv


In [10]:
df

,frame,track_id,kp0_x,kp0_y,kp0_conf,kp1_x,kp1_y,kp1_conf,kp2_x,kp2_y,...,kp13_conf,kp14_x,kp14_y,kp14_conf,kp15_x,kp15_y,kp15_conf,kp16_x,kp16_y,kp16_conf
0,1,1,2132.779053,478.932861,0.971162,2140.505615,467.270020,0.987464,2130.284912,469.569824,...,0.998786,2178.529541,781.894531,0.991195,2313.026855,895.240845,0.997021,2223.273438,898.117554,0.987450
1,1,2,904.149658,466.433960,0.349978,900.575806,456.085388,0.022987,903.796509,456.358704,...,0.959473,861.958679,782.340088,0.995629,838.714233,896.918701,0.918555,832.353088,900.541992,0.983290
2,2,1,2133.257324,479.305054,0.976934,2141.233643,467.651367,0.989269,2129.917725,469.936157,...,0.998750,2173.027100,782.845947,0.991643,2320.195557,898.740479,0.996987,2229.451416,896.494385,0.988079
3,2,2,905.758484,463.393555,0.525930,902.218567,452.651123,0.036250,903.475464,453.852417,...,0.944289,858.233215,784.067505,0.994778,837.657227,896.239502,0.901064,829.242432,904.542847,0.981439
4,3,1,2133.453613,479.285400,0.980820,2141.130127,467.504639,0.990218,2129.831787,469.639771,...,0.998787,2168.082764,783.222778,0.992233,2325.905762,903.075439,0.997032,2232.459717,894.149292,0.988689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1531,384,6,769.141846,366.693115,0.997168,800.544373,341.360291,0.995390,749.756958,347.836304,...,0.995795,844.402649,1154.964844,0.988469,813.694763,1411.941895,0.924722,950.532349,1345.521729,0.871645
1532,384,7,4.217270,498.780029,0.030369,7.298851,475.082642,0.015981,6.646355,477.847778,...,0.334970,0.000000,1131.788818,0.235341,118.665253,1256.528320,0.429173,115.739960,1251.168335,0.343665
1533,385,4,1694.104370,653.622803,0.996985,1705.246826,644.395996,0.991620,1684.704956,645.351562,...,0.998425,1663.392822,999.590210,0.998283,1805.897461,1120.635010,0.990033,1655.832764,1101.591309,0.989507
1534,385,6,755.238220,356.902222,0.997027,786.216431,330.455566,0.995082,736.071655,338.329590,...,0.994136,823.883545,1147.077881,0.988110,804.625977,1382.651367,0.913330,939.820923,1348.725342,0.876013
